# Installing Libraires

In [1]:
!pip install --quiet --upgrade langchain langchain-core langchain-community langchain-openai

# Loading ENV Varibales

In [2]:
import os
from dotenv import load_dotenv

_ = load_dotenv()

AZURE_OPENAI_API_ENDPOINT: str = os.environ.get('AZURE_OPENAI_API_ENDPOINT')
AZURE_OPENAI_API_KEY: str = os.environ.get('AZURE_OPENAI_API_KEY')
MODEL_NAME: str = os.environ.get('MODEL_NAME')
OPENAI_API_TYPE: str = os.environ.get('OPENAI_API_TYPE')
DEPLOYMENT_NAME: str = os.environ.get('DEPLOYMENT_NAME')
AZURE_OPENAI_API_VERSION: str = os.environ.get('AZURE_OPENAI_API_VERSION')

# Loading Azure Chat Model

In [3]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    model_name=MODEL_NAME,
    azure_endpoint=AZURE_OPENAI_API_ENDPOINT, 
    openai_api_key=AZURE_OPENAI_API_KEY, 
    openai_api_version=AZURE_OPENAI_API_VERSION, 
    openai_api_type=OPENAI_API_TYPE,
    azure_deployment=DEPLOYMENT_NAME,
    temperature=0,
    model_kwargs={"top_p": 0.1},
    )

# Explore the Model

Lets test the model we loaded

In [4]:
llm.invoke("Hello. How are u?")

AIMessage(content="Hello! I'm an AI language model, so I don't have feelings, but I'm here to help you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-c46496cd-04e8-422e-ac45-d8473a48fe65-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44})

In [5]:
llm_test_output = llm.invoke("Hello. My name is Ajit Emmanuel.")
print(llm_test_output.content)

Hello Ajit Emmanuel! How can I assist you today?


Example for translation from English to other languages

In [6]:
query = "Translate 'Hello My name is Ajit Emmanuel' to French"

llm.invoke(query).content

"Bonjour, je m'appelle Ajit Emmanuel."

In [7]:
from langchain_core.messages import SystemMessage, HumanMessage

In [8]:
message = HumanMessage(content=query)

llm.invoke([message]).content

"Bonjour, je m'appelle Ajit Emmanuel."

In [9]:
system_message = "Translate the sentence from English to German"
query = "My name is Ajit Emmanuel"

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content=query)
]

llm.invoke(messages).content

'Mein Name ist Ajit Emmanuel.'

In [10]:
system_message = "Translate the sentence from English to Spanish"
query = "My name is Ajit Emmanuel"

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content=query)
]

llm.invoke(messages).content

'Mi nombre es Ajit Emmanuel'

# Output Parser

In [11]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [12]:
output = llm.invoke(messages)
print(output)

content='Mi nombre es Ajit Emmanuel' response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 24, 'total_tokens': 30}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-ae1dd004-18e2-420e-b2a6-8d47080848c9-0' usage_metadata={'input_tokens': 24, 'output_tokens': 6, 'total_tokens': 30}


In [13]:
parser.invoke(output)

'Mi nombre es Ajit Emmanuel'

In [14]:
chain = llm | parser

chain.invoke(messages)

'Mi nombre es Ajit Emmanuel'

# Prompt Template

In [15]:
from langchain_core.prompts import ChatPromptTemplate

In [16]:
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [17]:
system_template = "Translate the sentence in english to following {language}"
messages = [("system", system_template), ("user", "{text}")]
prompt_template = ChatPromptTemplate.from_messages(messages)
print(prompt_template)

input_variables=['language', 'text'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translate the sentence in english to following {language}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]


In [18]:
query = {"language": "italian", "text": "Hello. My name is Ajit Emmanuel"}
prompt_result = prompt_template.invoke(query)
prompt_result

ChatPromptValue(messages=[SystemMessage(content='Translate the sentence in english to following italian'), HumanMessage(content='Hello. My name is Ajit Emmanuel')])

In [19]:
prompt_result.to_messages()

[SystemMessage(content='Translate the sentence in english to following italian'),
 HumanMessage(content='Hello. My name is Ajit Emmanuel')]

In [20]:
chain = prompt_template | llm |parser

chain.invoke(query)

'Ciao. Il mio nome è Ajit Emmanuel.'

# Fast Api App

In [21]:
!pip install --quiet --upgrade fastapi uvicorn nest_asyncio "langserve[all]"

In [22]:
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# Apply the nest_asyncio patch to the event loop
import nest_asyncio
nest_asyncio.apply()

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = llm

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    config = uvicorn.Config(app, host="127.0.0.1", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

INFO:     Started server process [4186]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)



 __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
|  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
|  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
|  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
|  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
|_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|

LANGSERVE: Playground for chain "/chain/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /chain/playground/
LANGSERVE:
LANGSERVE: See all available routes at /docs/

LANGSERVE: ⚠️ Using pydantic 2.7.2. OpenAPI docs for invoke, batch, stream, stream_log endpoints will not be generated. API endpoints and playground should work as expected. If you need to see the docs, you can downgrade to pydantic 1. For example, 